In [ ]:
import batoid
import yaml
import os
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
telescope = batoid.Optic.fromYaml("HSC.yaml")

In [ ]:
# Sanity check optic shifting
@interact(optic=widgets.Dropdown(
              options=['SubaruHSC.HSC.WFC.G1',
                       'SubaruHSC.HSC.WFC.G2',                       
                       'SubaruHSC.HSC.WFC.ADC',
                       'SubaruHSC.HSC.WFC.ADC.ADC1',
                       'SubaruHSC.HSC.WFC.ADC.ADC2',
                       'SubaruHSC.HSC.WFC.G3',
                       'SubaruHSC.HSC.WFC.G4',
                       'SubaruHSC.HSC.WFC.G5',
                       'SubaruHSC.HSC.CAM.F',
                       'SubaruHSC.HSC.CAM.W',
                       'SubaruHSC.HSC.CAM.D'],
              default='SubaruHSC.HSC.WFC.ADC'
          ),
          gdx=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.2),
          gdy=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
          gdz=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
          dthx=widgets.FloatSlider(min=-90, max=90, step=5, value=0.0),
          dthy=widgets.FloatSlider(min=-90, max=90, step=5, value=0.0),
          dthz=widgets.FloatSlider(min=-90, max=90, step=5, value=0.0),
          ldx=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
          ldy=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
          ldz=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
         )
def shiftOptic(optic, gdx, gdy, gdz, dthx, dthy, dthz, ldx, ldy, ldz):
    rot = batoid.RotZ(np.pi/180*dthz).dot(batoid.RotX(np.pi/180*dthx)).dot(batoid.RotY(np.pi/180*dthy))
    perturbedTelescope = telescope.withGloballyShiftedOptic(optic, [gdx, gdy, gdz])
    perturbedTelescope = perturbedTelescope.withLocallyRotatedOptic(optic, rot)
    perturbedTelescope = perturbedTelescope.withLocallyShiftedOptic(optic, [ldx, ldy, ldz])
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection='3d')
    perturbedTelescope['SubaruHSC.HSC'].draw3d(ax, c='k')
    #And redraw the perturbed optic in red
    perturbedTelescope[optic].draw3d(ax, c='r')
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(-0.5, 0.5)    
    ax.set_zlim(13.5, 15.25)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")    
    plt.show()